<a href="https://colab.research.google.com/github/dvpolyakov/DataVisualization/blob/master/ColabNoetbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import plotly

In [1]:
!wget "https://qed.qld.gov.au/det-publications/reports/Documents/annual-report/17-18/overseas-travel-2017-18.XLSX"

--2019-10-09 23:15:09--  https://qed.qld.gov.au/det-publications/reports/Documents/annual-report/17-18/overseas-travel-2017-18.XLSX
Resolving qed.qld.gov.au (qed.qld.gov.au)... 103.233.236.227
Connecting to qed.qld.gov.au (qed.qld.gov.au)|103.233.236.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38660 (38K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘overseas-travel-2017-18.XLSX’

overseas-travel-201 100%[===================>]  37.75K   185KB/s    in 0.2s    

2019-10-09 23:15:12 (185 KB/s) - ‘overseas-travel-2017-18.XLSX’ saved [38660/38660]



In [0]:
df = pd.read_excel("overseas-travel-2017-18.XLSX")
df.fillna(0, inplace=True)
types = list(map(str, map(type, df['Agency cost ($)'])))
numeric_classes = {"<class 'float'>", "<class 'int'>"}
t = [1 if x in numeric_classes else 0 for x in types]
df = df[np.array(t, dtype=np.bool)]
df['total_costs'] = df[df.columns[-2]].values + df[df.columns[-1]].values
df.drop(['Agency cost ($)', 'Contribution from External Sources ($)', "Name of officer / member and position"], axis=1, inplace=True)

In [0]:
df.reset_index(drop=True)
df.drop(df.index[[271]], inplace=True)
counts = df.groupby(["Reason for Travel"]).count().drop(['total_costs'], axis=1).to_dict()["Destination"]
total_costs = df.groupby(["Reason for Travel"]).sum().drop(['Destination'], axis=1).to_dict()["total_costs"]

In [0]:
costs_per_person = {}

for key in counts:
    costs_per_person[key] = int(total_costs[key] / counts[key])

In [6]:
import plotly.graph_objects as go

months = list(costs_per_person.keys())

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=list(costs_per_person.values()),
    name='Primary Product',
    marker_color='indianred'
))

fig.update_layout(
    title=go.layout.Title(
        text="Price of business trip for different types of educational events",
        xref="paper",
        x=0.5, y=0.87
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="US dollars",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(xaxis_tickangle=-30)
fig.show()